In [1]:
from anony_process_pipeline import *

In [151]:
from werkzeug.datastructures import FileStorage
import os

def import_subject_images(ids):
    image_folder = "../../data/database"
    sujet = f"subject_{ids}_"
    image_files = []
    for filename in os.listdir(image_folder):
        if filename.startswith(sujet):
            chemin_fichier = f"{image_folder}/{filename}"
            file = open(chemin_fichier, "rb")
            file_storage = FileStorage(stream=file, filename=filename)
            image_files.append(file_storage)
    return image_files

# Import images
img_list = import_subject_images(3)

#########################################################################################
##### Accepte list of FileStorage by default: reduce this block to 1 line & 3 words #####
#########################################################################################
from modules.utils_image import filestorage_image_to_pil
import pandas as pd
df = pd.DataFrame({'userFaces':filestorage_image_to_pil(img_list), 'subject_number':1, 'imageId':1})
res1 = run_preprocessing(df_images=df)

# Realise images in storage
for img_in_storage in img_list:
    img_in_storage.close()


[img['flattened_image'] for img in res1[list(res1.keys())[0]]]

[array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.15686275, 0.15686275,
        0.15686275]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.54117647, 0.60784314, 0.55686275, ..., 0.63921569, 0.61176471,
        0.58431373]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
        0.73333333]),
 array([0.81960784, 0.81960784, 0.81960784, ..., 0

In [152]:
#########################################################################################
##### Aurait pu se resume to : function(images, n_components_ratio)
#########################################################################################
n_components_ratio = 0.9
user = list(res1.keys())[0]
images = res1[user] # keys: (resized_image, grayscale_image, normalized_image, flattened_image)
flattened_stack = np.array([img['flattened_image'] for img in images])
n_components = min(int(n_components_ratio * flattened_stack.shape[0]), flattened_stack.shape[1])

print("n_components:", n_components)
res2 = run_eigenface(flattened_stack, 4)
pca, eigenfaces, mean_face, projection = res2 # eigenfaces save to db

n_components: 9


In [154]:
# Étape 3 : Ajout de bruit différentiel
epsilon = 0.1
noised_projection = run_add_noise(projection, epsilon, sensitivity=1.0)
# Étape 4 : Reconstruction
reconstructed_images = run_reconstruction(pca, noised_projection)

In [156]:
import base64
from io import BytesIO
from PIL import Image

imgs = [Image.open(BytesIO(base64.b64decode(img))) for img in reconstructed_images]
print(len(imgs))
imgs[0].show()

11
